In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [2]:
ex = pd.read_csv('/content/megaGymDataset.csv')


#Understanding the Data

##Exercises DataSet

In [3]:
ex.columns

Index(['Unnamed: 0', 'Title', 'Desc', 'Type', 'BodyPart', 'Equipment', 'Level',
       'Rating', 'RatingDesc'],
      dtype='object')

In [4]:
ex.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [5]:
ex = ex.rename(columns={'Unnamed: 0': 'ExNo'})

In [6]:
ex.shape

(2918, 9)

In [7]:
ex.isna().sum()

,0
ExNo,0
Title,0
Desc,1550
Type,0
BodyPart,0
Equipment,32
Level,0
Rating,1887
RatingDesc,2056


In [8]:
ex.Level.value_counts()

,count
Level,
Intermediate,2446
Beginner,459
Expert,13


In [9]:
ex.Type.value_counts()

,count
Type,
Strength,2545
Stretching,147
Plyometrics,97
Powerlifting,37
Cardio,35
Olympic Weightlifting,35
Strongman,22


In [10]:
ex.BodyPart.value_counts()

,count
BodyPart,
Abdominals,662
Quadriceps,646
Shoulders,340
Chest,262
Biceps,168
Triceps,151
Lats,124
Hamstrings,121
Middle Back,118


In [11]:
ex_for_analysis = ex.copy()

In [12]:
ex.columns

Index(['ExNo', 'Title', 'Desc', 'Type', 'BodyPart', 'Equipment', 'Level',
       'Rating', 'RatingDesc'],
      dtype='object')

In [13]:
ex_for_analysis = ex.drop(['Title',  'Desc', 'Rating', 'RatingDesc', 'Desc', 'Equipment'], axis=1)

In [14]:
ex_for_analysis.columns

Index(['ExNo', 'Type', 'BodyPart', 'Level'], dtype='object')

#Synthetically generating data


In [15]:
random.seed(42)

In [16]:
fitness_goal_to_exercise_type = {
    'Weight loss': ['Cardio', 'Strength', 'HIIT'],
    'Muscle gain': ['Strength', 'Powerlifting', 'Olympic Weightlifting'],
    'Endurance': ['Cardio', 'Plyometrics', 'Strength'],
    'General fitness': ['Strength', 'Cardio', 'Yoga', 'Plyometrics']
}

secondary_goal_to_exercise_type = {
    'Flexibility': ['Stretching', 'Yoga', 'Pilates'],
    'Balance': ['Yoga', 'Pilates', 'Strength'],
    'Stress reduction': ['Yoga', 'Stretching', 'Cardio'],
    'Posture correction': ['Strength', 'Yoga', 'Stretching'],
    'Agility': ['Plyometrics', 'Strength', 'Cardio']
}

fitness_goal_to_secondary_goal = {
    'Weight loss': ['Flexibility', 'Balance', 'Stress reduction'],
    'Muscle gain': ['Flexibility', 'Agility', 'Posture correction'],
    'Endurance': ['Stress reduction', 'Flexibility', 'Balance'],
    'General fitness': ['Flexibility', 'Balance', 'Stress reduction']
}

exercise_type_to_body_parts = {
    'Strength': ['Chest', 'Biceps', 'Triceps', 'Shoulders', 'Quadriceps', 'Hamstrings', 'Glutes'],
    'Stretching': ['Shoulders', 'Hamstrings', 'Lower Back', 'Glutes', 'Abdominals'],
    'Plyometrics': ['Legs', 'Glutes', 'Core'],
    'Powerlifting': ['Chest', 'Shoulders', 'Legs', 'Back'],
    'Cardio': ['Legs', 'Full body'],
    'Olympic Weightlifting': ['Shoulders', 'Legs', 'Back'],
    'Strongman': ['Full body', 'Legs', 'Back', 'Shoulders']
}


In [17]:
ex_type_dict = ex_for_analysis.groupby('Type')['ExNo'].apply(list).to_dict()
for i in ex_type_dict:
  print(f'{i}: {ex_type_dict[i]}')


Cardio: [149, 349, 884, 993, 1337, 1729, 2000, 2002, 2030, 2031, 2066, 2068, 2071, 2075, 2076, 2077, 2078, 2118, 2123, 2124, 2130, 2134, 2159, 2165, 2167, 2169, 2171, 2175, 2176, 2177, 2191, 2207, 2235, 2738, 2739]
Olympic Weightlifting: [1217, 1293, 1295, 1296, 1297, 1298, 1300, 1301, 1302, 1313, 1314, 1315, 1753, 1754, 1798, 1801, 1802, 1803, 1805, 1816, 1820, 1821, 1825, 1827, 1831, 1834, 1841, 1842, 1846, 1847, 2449, 2464, 2465, 2467, 2469]
Plyometrics: [96, 196, 232, 248, 623, 624, 668, 676, 681, 683, 971, 976, 980, 985, 988, 991, 1098, 1099, 1102, 1103, 1104, 1158, 1164, 1165, 1168, 1331, 1335, 1336, 1341, 1369, 1370, 1373, 1380, 1382, 1516, 1528, 1529, 1881, 1921, 1928, 1933, 1998, 2006, 2008, 2015, 2017, 2019, 2021, 2022, 2023, 2024, 2025, 2120, 2122, 2125, 2126, 2129, 2131, 2132, 2136, 2137, 2138, 2139, 2140, 2153, 2154, 2155, 2157, 2158, 2160, 2161, 2162, 2168, 2170, 2172, 2173, 2174, 2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2192, 2414, 2418, 2421, 2424, 2735, 2740, 27

##Synthetic Generation of user profile

In [18]:
genders = ['Male', 'Female']
fitness_goals = ['Weight loss', 'Muscle gain', 'Endurance', 'General fitness']
secondary_goals = ['Flexibility', 'Balance', 'Stress reduction', 'Posture correction', 'Agility']
activity_types = ['Cardio', 'Strength training', 'Yoga', 'Walking', 'Cycling', 'Swimming', 'HIIT']
workout_intensities = ['Beginner', 'Intermediate', 'Expert']

In [19]:
def generate_user_profile(user_id):
    age = random.randint(18, 35)
    gender = random.choice(genders)
    height = random.randint(150, 190)  # Height in cm
    weight = random.randint(45, 100)  # Weight in kg
    bmi = round(weight / ((height / 100) ** 2), 2)  # Calculate BMI using weight and height

    # Generate fitness goals and preferences
    primary_goal = random.choice(fitness_goals)
    secondary_goal = random.choice(secondary_goals)
    preferred_activity = random.choice(activity_types)
    workout_intensity = random.choice(workout_intensities)

    # Build the user profile with a unique user_id
    user_profile = {
        'uid': user_id,
        'Age': age,
        'Gender': gender,
        'Height (cm)': height,
        'Weight (kg)': weight,
        'BMI': bmi,
        'Primary Fitness Goal': primary_goal,
        'Secondary Fitness Goal': secondary_goal,
        'Preferred Activity Type': preferred_activity,
        'Workout Intensity': workout_intensity,
    }

    return user_profile

# Generate a list of user profiles with random user IDs (from 1 to 10)
user_profiles = pd.DataFrame([generate_user_profile(user_id) for user_id in range(1001, 1011)])

# Print the generated user profiles
print(user_profiles)


    uid  Age  Gender  Height (cm)  Weight (kg)    BMI Primary Fitness Goal  \
0  1001   21    Male          167           60  21.51          Muscle gain   
1  1002   35    Male          187           72  20.59          Weight loss   
2  1003   25    Male          185           57  16.65      General fitness   
3  1004   26    Male          160           89  34.77      General fitness   
4  1005   24  Female          156           50  20.55      General fitness   
5  1006   26    Male          179           79  24.66          Weight loss   
6  1007   27  Female          186           57  16.48          Weight loss   
7  1008   27    Male          164          100  37.18          Weight loss   
8  1009   29    Male          173           67  22.39          Muscle gain   
9  1010   20    Male          184           91  26.88          Muscle gain   

  Secondary Fitness Goal Preferred Activity Type Workout Intensity  
0                Balance                Swimming          Beginner  
1  

##Synthetic generation of workout feedback



In [20]:
print(user_profiles)
print(ex_for_analysis.head())
print(fitness_goal_to_exercise_type)

    uid  Age  Gender  Height (cm)  Weight (kg)    BMI Primary Fitness Goal  \
0  1001   21    Male          167           60  21.51          Muscle gain   
1  1002   35    Male          187           72  20.59          Weight loss   
2  1003   25    Male          185           57  16.65      General fitness   
3  1004   26    Male          160           89  34.77      General fitness   
4  1005   24  Female          156           50  20.55      General fitness   
5  1006   26    Male          179           79  24.66          Weight loss   
6  1007   27  Female          186           57  16.48          Weight loss   
7  1008   27    Male          164          100  37.18          Weight loss   
8  1009   29    Male          173           67  22.39          Muscle gain   
9  1010   20    Male          184           91  26.88          Muscle gain   

  Secondary Fitness Goal Preferred Activity Type Workout Intensity  
0                Balance                Swimming          Beginner  
1  

In [21]:
def generate_workoutfeedback(user_profiles, ex_type_dict, fitness_goal_to_exercise_type):
    workout_feedback_data = []
    for _, user in user_profiles.iterrows():
        primary_goal = user['Primary Fitness Goal']
        relevant_exercise_types = fitness_goal_to_exercise_type.get(primary_goal, [])
        relevant_exercises = []
        for exercise_type in relevant_exercise_types:
            relevant_exercises.extend(ex_type_dict.get(exercise_type, []))
        selected_exercises = random.sample(relevant_exercises, k=random.randint(1, 3))  # Select 1 to 3 exercises randomly

        for exid in selected_exercises:
            feedback_entry = {
                'uid': user['uid'],
                'exid': exid,
                'frequency': random.choice(['Daily',
                     '5 times a week',
                     '3 times a week',
                     'Twice a week',
                     'Once a week']
),
                'completionstatus': random.choice(['Completed', 'Skipped', 'Not Attempted']),
                'rating': random.randint(1, 5),  # Rating scale from 1 to 5
                'motivation_level': random.randint(1, 5)  # Motivation level from 1 to 5
            }
            workout_feedback_data.append(feedback_entry)

    workoutfeedback = pd.DataFrame(workout_feedback_data)
    return workoutfeedback

workoutfeedback = generate_workoutfeedback(user_profiles, ex_type_dict, fitness_goal_to_exercise_type)

print(workoutfeedback)


     uid  exid       frequency completionstatus  rating  motivation_level
0   1001  2613  3 times a week        Completed       2                 1
1   1001   956  3 times a week          Skipped       3                 1
2   1002  2620  3 times a week        Completed       4                 4
3   1003  2196  5 times a week        Completed       5                 5
4   1003   601  3 times a week    Not Attempted       5                 4
5   1003  1153     Once a week          Skipped       3                 2
6   1004  2407    Twice a week        Completed       1                 1
7   1005  2169  5 times a week    Not Attempted       4                 5
8   1006  1728    Twice a week    Not Attempted       4                 5
9   1007  2563           Daily    Not Attempted       5                 3
10  1007    12  3 times a week        Completed       3                 4
11  1008  2079           Daily    Not Attempted       3                 5
12  1009  2399           Daily    Not 

In [22]:
user_profiles.to_csv('user_profiles.csv', index=False)
workoutfeedback.to_csv('workoutfeedback.csv', index=False)

In [23]:
'''from google.colab import files
files.download('user_profiles.csv')
files.download('workoutfeedback.csv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>